In [1]:
from huggingface_hub import login
login(token="---")

In [2]:
import gdown
import zipfile

url = f"https://drive.google.com/uc?id=1BJqn1s7J2LAMDenQ7FaORrcueAEvpm0K"

output = "dataset.zip"
gdown.download(url, output, quiet=False)

with zipfile.ZipFile(output, "r") as z:
    z.extractall("dataset")

Downloading...
From (original): https://drive.google.com/uc?id=1BJqn1s7J2LAMDenQ7FaORrcueAEvpm0K
From (redirected): https://drive.google.com/uc?id=1BJqn1s7J2LAMDenQ7FaORrcueAEvpm0K&confirm=t&uuid=89a0adf7-1824-488a-9f18-c69c8fe996f1
To: /kaggle/working/dataset.zip
100%|██████████| 1.18G/1.18G [00:05<00:00, 217MB/s]


In [3]:
import os
from datasets import Dataset, Features, Image as HFImage, Value

thermal_dir = "dataset/thermal_8_bit"
rgb_dir = "dataset/RGB"

names = sorted([
    f for f in os.listdir(thermal_dir)
    if f.lower().endswith((".png", ".jpg", ".jpeg"))
])[:1000]

In [4]:
def gen():
    for fname in names:
        print(fname)
        thermal_path = os.path.join(thermal_dir, fname)
        rgb_path = os.path.join(rgb_dir, fname).replace(".jpeg", ".jpg")

        yield {
            "conditioning_image": thermal_path,
            "image": rgb_path,
            "prompt": "",
        }

features = Features({
    "conditioning_image": HFImage(),
    "image": HFImage(),
    "prompt": Value("string"),
})

dataset = Dataset.from_generator(gen, features=features)

dataset.push_to_hub("swetha3456/thermal-rgb-dataset")


In [5]:
!pip install git+https://github.com/huggingface/diffusers.git transformers accelerate xformers==0.0.16 datasets==2.21.0

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-_3tsvsuo
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-_3tsvsuo
  Resolved https://github.com/huggingface/diffusers.git to commit 0c1ccc0775519d23a01732d62cca202215e8956b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 75.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
# download the official ControlNet training script
!wget https://raw.githubusercontent.com/huggingface/diffusers/main/examples/controlnet/train_controlnet.py -O train_controlnet.py

--2025-12-15 09:34:24--  https://raw.githubusercontent.com/huggingface/diffusers/main/examples/controlnet/train_controlnet.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48801 (48K) [text/plain]
Saving to: ‘train_controlnet.py’

train_controlnet.py 100%[===================>]  47.66K  --.-KB/s    in 0.007s  

2025-12-15 09:34:24 (6.74 MB/s) - ‘train_controlnet.py’ saved [48801/48801]



In [7]:
!pip install -U peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: peft
    Found existing installation: peft 0.16.0
    Uninstalling peft-0.16.0:
      Successfully uninstalled peft-0.16.0


In [8]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 32.8 MB/s eta 0:00:00


In [10]:
!accelerate launch train_controlnet.py \
 --pretrained_model_name_or_path="swetha3456/thermal-rgb-controlnet-v2" \
 --revision="checkpoint-5000-contrast-x2" \
 --output_dir="thermal-rgb-controlnet-v3-contrast" \
 --dataset_name=swetha3456/thermal-rgb-dataset-v3-contrast \
 --conditioning_image_column=conditioning_image \
 --image_column=image \
 --caption_column=prompt \
 --resolution=384 \
 --mixed_precision=fp16 \
 --learning_rate=3e-5 \
 --lr_scheduler=constant_with_warmup \
 --lr_warmup_steps=500 \
 --max_train_steps=5000 \
 --validation_image "dataset/thermal_8_bit/FLIR_02280.jpeg" "dataset/thermal_8_bit/FLIR_02278.jpeg" \
 --validation_prompt "outdoor street scene with trees and road" "road with trees and vegetation" \
 --train_batch_size=4 \
 --gradient_accumulation_steps=4 \
 --gradient_checkpointing \
 --use_8bit_adam \
 --set_grads_to_none \
 --enable_xformers_memory_efficient_attention \
 --checkpointing_steps=1000 \
 --validation_steps=100 \
 --seed=42 \
 --tracker_project_name=controlnet-v2 \
 --push_to_hub

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 